In [46]:
import urllib.request
import urllib.error
import os, sys
import urllib
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml.html as lh
import numpy as np
import re

In [ ]:
pip install requests
pip install beautifulsoup4
pip install --user pipenv
pip install numpy

In [47]:
response = requests.get('https://www.ncwater.org/Drought_Monitoring/statusReport.php/')
stored_contents = lh.fromstring(response.content)
table_elements = stored_contents.xpath('//tr')
[len(T) for T in table_elements[:12]]

[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [48]:
#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in table_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 PWSID
2 Water System
3 Status
4 Reason for Restriction
5 County
6 Date of Last Update
7 Latest WSRP


In [49]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(table_elements)):
    #T is our j'th row
    T=table_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=7:
        break
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [50]:
[len(C) for (title,C) in col]

[539, 539, 539, 539, 539, 539, 539]

In [51]:
Dict ={title:column for (title,column) in col}
Newest_Updates =pd.DataFrame(Dict)

In [52]:
column_names = ["PWSID", "Water System", "Status", "Reason for Restriction", "County", "Date of Last Update", "Latest WSRP"]
Updates = pd.DataFrame(columns = column_names)
print(Updates)

Empty DataFrame
Columns: [PWSID, Water System, Status, Reason for Restriction, County, Date of Last Update, Latest WSRP]
Index: []


In [53]:
Newest_Updates = Newest_Updates[~Newest_Updates['PWSID'].astype(str).str.startswith('PWSID')]
Newest_Updates[0:20]

,PWSID,Water System,Status,Reason for Restriction,County,Date of Last Update,Latest WSRP
0,04-71-111,BELVEDERE PLANTATION,Voluntary,Drought,PENDER,2020-02-13,2018-08-21
1,01-44-015,Canton,Voluntary,Drought,HAYWOOD,2020-06-05,2018-10-29
2,03-43-010,Dunn,Not Restricted,,HARNETT,2020-05-04,2018-10-19
3,03-32-010,Durham,Mandatory,Year Round,DURHAM,2020-06-10,2017-02-08
4,04-25-132,Fairfield Harbour,Voluntary,Drought,CRAVEN,2020-02-13,2018-08-24
5,03-26-035,Falcon,,,CUMBERLAND,2020-05-04,2018-12-17
7,02-01-035,Alamance,Not Restricted,,ALAMANCE,2019-06-05,2011-02-02
8,01-02-020,Alexander County WDs,Not Restricted,,ALEXANDER,2019-06-04,2018-10-04
9,03-04-010,Anson County,Not Restricted,,ANSON,2019-06-06,2019-08-19
10,04-16-035,Atlantic Beach,Not Restricted,,CARTERET,2019-06-05,2018-05-15


In [54]:
html_page = urlopen("https://www.ncwater.org/Drought_Monitoring/statusReport.php")
soup = BeautifulSoup(html_page)

ev_list = []
for tag in soup.find_all('tr'):
    Answer = tag.find_all('a')
    if Answer != []:
        ev_list.append(Answer[0]['href'])
    else:
        pass
    
Link_Dataframe = pd.DataFrame(ev_list)
Link_Dataframe.columns = ["External Links"]

In [55]:
Bigger_Dataframe = pd.merge(Newest_Updates, Link_Dataframe, left_index=True, right_index=True)
Bigger_Dataframe

,PWSID,Water System,Status,Reason for Restriction,County,Date of Last Update,Latest WSRP,External Links
0,04-71-111,BELVEDERE PLANTATION,Voluntary,Drought,PENDER,2020-02-13,2018-08-21,https://www.ncwater.org/Water_Supply_Planning/...
1,01-44-015,Canton,Voluntary,Drought,HAYWOOD,2020-06-05,2018-10-29,https://www.ncwater.org/Water_Supply_Planning/...
2,03-43-010,Dunn,Not Restricted,,HARNETT,2020-05-04,2018-10-19,https://www.ncwater.org/Water_Supply_Planning/...
3,03-32-010,Durham,Mandatory,Year Round,DURHAM,2020-06-10,2017-02-08,https://www.ncwater.org/Water_Supply_Planning/...
4,04-25-132,Fairfield Harbour,Voluntary,Drought,CRAVEN,2020-02-13,2018-08-24,https://www.ncwater.org/Water_Supply_Planning/...
...,...,...,...,...,...,...,...,...
514,03-83-020,Wagram,Not Restricted,,SCOTLAND,2011-05-03,2019-01-25,https://www.ncwater.org/Water_Supply_Planning/...
515,04-96-155,Walnut Creek,Not Restricted,,WAYNE,2011-05-03,2019-03-22,https://www.ncwater.org/Water_Supply_Planning/...
516,04-40-030,Walstonburg,Not Restricted,,GREENE,2011-05-03,2018-08-15,https://www.ncwater.org/Water_Supply_Planning/...
517,02-93-015,Warren County,Not Restricted,,WARREN,2011-05-10,2018-06-11,https://www.ncwater.org/Water_Supply_Planning/...
